Optimizes point with chamfer, distance to vstars, repulsion. Reconstruction using mean normal and taku itoh. Kind of works excpet for sparse sampling / many details 

rvd_dual:
- the "networks" outputs positions p, normals n and quadrics q  (from the voronoi region)
- Compute v*
- Build the Delaunay/Voronoi of v*
- Select all tets which circumcenter are inside the 4 half spaces defined by v*(ind) and n(ind)
- output boundary of tets



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../utils/')
import neural_quadrics as nq
import mesh_tools as mt
import torch
from torch import nn
from tqdm import tqdm
from pytorch3d.loss import chamfer_distance
from meshplot import plot
from IPython.display import clear_output
import os
import trimesh
from pytorch3d.ops import knn_points
from direct import *


In [ ]:
GRID_N = 32

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model_name = '../../data/thingy32/groundtruths/441708.stl' #bunny
# model_name = '../../data/thingy32/groundtruths/527631.stl'
model_name = '../../data/thingy32/groundtruths/95444.stl'
model_name = '../../data/thingy32/groundtruths/47984.stl'
# model_name = '/data/nmaruani/DATASETS/fun/spearman.stl'

# model_name = '/data/nmaruani/DATASETS/fun/ActionChess_-_Pawn_B_x6.stl'
# model_name = '/data/nmaruani/DATASETS/fun/cubeminus.obj'

input_points, input_normals = mt.load_shape(model_name, normalize='NDC', sample_n=int(1.5e5*GRID_N**2/32**2))



In [ ]:
points = mt.mesh_grid(GRID_N, True)
V = nq.MovingQuadrics(points[mt.mask_relevant_voxels(GRID_N, input_points)], device)

optimizer = torch.optim.Adam([V.points], 1e-3/(GRID_N/32))


tensor_surface = torch.tensor(input_points, dtype=torch.float32).to(device)
tensor_normals = torch.tensor(input_normals, dtype=torch.float32).to(device)
L=[]


### Further optim

In [ ]:
def train_simple(V, optimizer, tensor_surface, repulsion_fac=0, sample_fac=1):
    optimizer.zero_grad()
    masks = torch.rand_like(tensor_surface[:, 0]) < sample_fac
    loss = chamfer_distance(
        tensor_surface[masks][None, :], V.points[None, :])[0].mean()
    if repulsion_fac > 0:
        min_dist = knn_points(V.points[None, :], V.points[None, :], K=2).dists[0, :, 1]**2
        loss += -repulsion_fac * min_dist.mean()
    x = loss.item()
    loss.backward()
    optimizer.step()
    return x


In [ ]:
for i in tqdm(range(200)):
    L.append((train_simple(V, optimizer, tensor_surface, repulsion_fac=0, sample_fac=.1)))
    
        # scheduler.step()
plt.plot(L)
plt.yscale('log')

### RVD dual Mesh 

In [ ]:
V.cluster_samples_quadrics_normals(tensor_surface, tensor_normals)

In [ ]:
plot(*V.quadric_ellipse_mesh())

In [ ]:
clear_output()
nv, nf = V.min_cut_surface(32)

mp = plot(nv,nf, shading={'wireframe': True})
# mp.add_mesh(*V.quadric_ellipse_mesh())
# meshplot_add_points(mp, voronoi.vertices)

In [ ]:
nv.min()

In [ ]:
# export_obj(*rvd_dual(V, tensor_surface, tensor_normals), 'pn_bunny')

### Batch RVD

In [ ]:
def train_simple(V, repulsion_fac=0, sample_fac=1):
    optimizer.zero_grad()
    masks = torch.rand_like(tensor_surface[:, 0]) < sample_fac
    loss = chamfer_distance(tensor_surface[masks][None,:], V.points[None,:])[0].mean()
    if repulsion_fac>0:
        loss += -repulsion_fac*(V.distance_to_centroids(V.points,V.points).topk(2, 0, False).values[1].mean())

    # quadrics = V.cluster_samples_quadrics(tensor_surface, tensor_normals)
    # vstars, _, _, mask_exist = vstars_from_quadrics(quadrics, V.points)
    # loss += ((V.points[mask_exist]-vstars)**2).sum(-1).mean()
    x = loss.item()
    loss.backward()
    optimizer.step() 
    return x


In [ ]:
GRID_N = 128

src_dir = '../../data/thingy32/groundtruths/'
for model_name in tqdm(os.listdir(src_dir)):
    v, f = mt.load_and_sample_shape(model_name, src_dir, 0, 'NDC')
    ref_mesh = trimesh.Trimesh(v,f)
    samples, face_index = trimesh.sample.sample_surface_even(ref_mesh, int(1e5*GRID_N**2/32**2))
    samples = np.array(samples)
    normals = np.array(ref_mesh.face_normals[face_index])

    points = mt.mesh_grid(GRID_N, True)
    V = nq.MovingQuadrics(points[mt.mask_relevant_voxels(GRID_N, samples)], device)
    global optimizer
    optimizer = None
    def reset_optimizer(lr=.005):
        global optimizer
        optimizer = torch.optim.Adam([V.points], lr)
    reset_optimizer()

    tensor_surface = torch.tensor(samples, dtype=torch.float32).to(device)
    tensor_normals = torch.tensor(normals, dtype=torch.float32).to(device)
    for i in (range(200)):
        train_simple(V, repulsion_fac=0, sample_fac=1)
    V.cluster_samples_quadrics_normals(tensor_surface, tensor_normals)
    torch.save(V, 'rvd_dual_{}/{}.pt'.format(GRID_N, model_name[:-4]))

### Evaluate

In [ ]:
V = torch.load('rvd_dual_64/44234.pt')
plot(*V.rvd_dual())

### Edge flip

In [ ]:
plot(*V.quadric_ellipse_mesh(size=0.02, lambd=2))

In [ ]:
edges = igl.edges(nf)
mid_points = np.ones((len(edges), 4))
mid_points[:, :3] = nv[edges].mean(1)

In [ ]:
mid_score = (mid_points[:, None, :]@quadrics[edges[:, 0]].cpu().detach().numpy()@mid_points[:, :, None]).squeeze()
mid_score += (mid_points[:, None, :]@quadrics[edges[:, 1]].cpu().detach().numpy()@mid_points[:, :, None]).squeeze()

### Configs

In [ ]:
with open('../configs/direct_thingi.yaml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.Loader)


cfg['io']['grid_n'] = 64
cfg['path']['out_dir'] = cfg['path']['out_dir'].format(
    cfg['io']['grid_n'])
cfg['io']['sample_n'] = int(
    cfg['io']['sample_n_base'] * (cfg['io']['grid_n']/32)**2)
cfg['optim']['lr'] = cfg['optim']['lr_base']/(cfg['io']['grid_n']/32)
L = []

In [ ]:
model_path = '../../data/thingy32/groundtruths/47984.stl'

input_points, input_normals = mt.load_shape(model_path, cfg['io']['in_pointcloud'], cfg['io']['normalize'], cfg['io']['sample_n'])

if cfg['model']['init'] == 'farthest' or not (32 is None):
    input_pc = input_points
else:
    input_pc = None

V = initialize_model(cfg['model']['n_points'],
                        cfg['optim']['device'], input_pc, cfg['io']['grid_n'])

optimizer = torch.optim.Adam([V.points], cfg['optim']['lr'])

tensor_surface = torch.tensor(
    input_points, dtype=torch.float32).to(cfg['optim']['device'])
tensor_normals = torch.tensor(
    input_normals, dtype=torch.float32).to(cfg['optim']['device'])

for _ in tqdm(range(cfg['optim']['epochs'])):
    L.append(train_simple(
        V, optimizer, tensor_surface, repulsion_fac=cfg['optim']['repulsion_fac'], sample_fac=cfg['optim']['sample_fac']))

# RVD dual Mesh
V.cluster_samples_quadrics_normals(tensor_surface, tensor_normals)

# # Save output
# save_output(V, cfg['path']['out_dir'], name[:-4], cfg['io']
#             ['out_pt'], cfg['io']['out_rvd'], cfg['io']['out_poisson'])

In [ ]:
clear_output()
nv, nf = V.rvd_dual()
mp = plot(nv,nf, shading={'wireframe': True})
# mp.add_mesh(*V.quadric_ellipse_mesh())
# meshplot_add_points(mp, voronoi.vertices)

In [ ]:
plt.plot(L)

In [ ]:
input_pc.shape